In [1]:
#imports
import pytorch_lightning as pl
import h5py
import os
import pandas as pd
import numpy as np
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import library as lib

# Set seed

In [2]:
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Load, Format, Split into train/validation/test datasets

In [3]:
#make numpy file with (seq header, embedding)
esm2 = h5py.File("./KLK_esm2.h5", 'r')
list(esm2.keys())
# List all groups
keys = list(esm2.keys())
#print(keys) 
# Get the data
embedding = []
headers = []
for key in keys:
    #data
    emb = esm2[key][:]
    #print(emb)
    #value = np.array(esm2[key][:])
    #print(value)
    embedding.append(emb)
    #headers
    key = key.replace("isoform=", "isoform_").replace("=", "").replace(":", "_")
    headers.append(key)

headers = np.array(headers)
embedding = np.array(embedding)

#split train test
headers_first, headers_test, embedding_first, embedding_test = train_test_split(headers, embedding, test_size=0.1, random_state=42)
#split train validation
headers_train, headers_validate, embedding_train, embedding_validate = train_test_split(headers_first, embedding_first, test_size =0.2, random_state=42)
print("total:", headers.shape)
print("training+validation:", headers_first.shape)
print("test:", headers_test.shape)
print("training:", headers_train.shape)
print("validation:", headers_validate.shape)
print(len(headers_train), len(embedding_train))
print(len(headers_validate), len(embedding_validate))

#training_df = pd.DataFrame({'header': headers_train, 'embedding' : embedding_train})
#validate_df = pd.DataFrame({'header': headers_validate, 'embedding' : embedding_validate})

total: (437,)
training+validation: (393,)
test: (44,)
training: (314,)
validation: (79,)
314 314
79 79


# Dataloaders

In [4]:
#dataloaders
train_dataset = lib.makedataset(headers_train, embedding_train)
val_dataset = lib.makedataset(headers_validate, embedding_validate)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0, drop_last=True)

for i in train_dataset:
    print(i[0])
    print(i[1])
    

[-0.04929698 -0.00028518 -0.02542126 ... -0.01088252 -0.14850137
 -0.01801342]
Alsi_B_02
[-0.06580624 -0.00940779 -0.03865383 ... -0.01996162 -0.1386906
 -0.0211166 ]
PRSS2_Canis_lupus
[-0.04963272 -0.02983764 -0.05298102 ... -0.02949603 -0.17247401
 -0.01377446]
Euma8
[-0.06824309  0.01601605 -0.01492161 ... -0.03785554 -0.11270773
 -0.03700221]
Alligator_sinensis_GeneID102385253
[-0.038883   -0.00737394 -0.03930339 ... -0.02110463 -0.16218737
  0.01818913]
Pelodiscus_sinensis_GeneID102446786
[-0.03669151  0.01628835 -0.01845741 ...  0.01185871 -0.14377901
 -0.02724892]
Xetr_out3
[-0.05436626  0.05898696 -0.02760512 ...  0.02091188 -0.16478686
 -0.00597407]
Mcpt4_Mus_musculus_GeneID17227
[-0.02974112 -0.02745553 -0.05608886 ...  0.00693845 -0.1375478
 -0.05972879]
Phci_A_07
[-0.06086486 -0.02988201 -0.00189185 ... -0.04371192 -0.15760238
 -0.04284367]
Vako_A_04
[-0.03130944  0.05178377 -0.03749208 ...  0.01469039 -0.14356282
 -0.02379861]
Gzmg_Mus_musculus_GeneID14944
[-0.07169278 -0.

# Training parameters

In [ ]:
#encoder_layers, latent_dim, decoder_layers
dim = 2560
a = int(dim * 1/16)
b = int(dim * 1/8)
c = int(dim * 1/2)
encoder_layers = [dim, c, b, a]
latent_dim = a
decoder_layers = [a, b, c, dim]

# Initialize the VAE model
vae = lib.VariationalAutoencoder(encoder_layers, latent_dim, decoder_layers)

#trainer = pl.Trainer(accelerator="auto", max_epochs=200)
trainer = pl.Trainer(accelerator="gpu", devices = 1, max_epochs=200)

# Train the VAE model
trainer.fit(vae, train_dataloader, val_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 3.7 M 
1 | latent  | Sequential | 26.1 K
2 | fc21    | Linear     | 25.8 K
3 | fc22    | Linear     | 25.8 K
4 | fc3     | Se

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (39) is smaller than the logg

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
#Validate
trainer.validate(vae, val_dataloader)

In [ ]:
#Plot



